In [1]:
# Lab 5 Logistic Regression Classifier
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

torch.manual_seed(777)

In [2]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
x = np.loadtxt('./firstLastData.csv', delimiter=',', dtype=np.float32)
y = np.loadtxt('./firstLastResult.csv', delimiter=',', dtype=np.float32)
x_data = x
y_data = y

In [4]:
len(y_data[y_data==1])

12406

In [5]:
X = Variable(torch.from_numpy(x_data))
Y = Variable(torch.from_numpy(y_data))

In [6]:
X.size()

torch.Size([26039, 259])

In [7]:
# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))

#linear = torch.nn.Linear(259, 1, bias=True)
#sigmoid = torch.nn.Sigmoid()
#model = torch.nn.Sequential(linear, sigmoid)




In [8]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(259, 128, bias=True)
        self.linear2 = torch.nn.Linear(128, 64, bias=True)
        self.linear3 = torch.nn.Linear( 64, 32, bias= True)
        self.linear4 = torch.nn.Linear( 32, 16, bias= True)
        self.linear5 = torch.nn.Linear( 16, 1, bias= True)
        
        self.linearTemp = torch.nn.Linear(128,1, bias=True)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.bn3 = torch.nn.BatchNorm1d(32)
        self.bn4 = torch.nn.BatchNorm1d(16)
        
        self.sig1 = torch.nn.Sigmoid()
        self.sig2 = torch.nn.Sigmoid()
        self.sig3 = torch.nn.Sigmoid()       
        self.sig4 = torch.nn.Sigmoid()
        self.sig5 = torch.nn.Sigmoid()
    
    def forward(self,x):
        x1 = self.bn1(F.relu(self.linear1(x)))
        x2 = self.bn2(F.relu(self.linear2(x1)))
        x3 = self.bn3( F.relu(self.linear3(x2)))
        x4 = self.bn4( F.relu(self.linear4(x3)))
        x5 = F.relu( self.linear5(x4))
        
        return x5
    
    def forwardSig(self,x):
        x1 = self.bn1( self.sig1(self.linear1(x)))
        x2 = self.bn2( self.sig2(self.linear2(x1)))
        x3 = self.bn3( self.sig3(self.linear3(x2)))
        x4 = self.bn4( self.sig4(self.linear4(x3)))
        x5 = self.sig5( self.linear5(x4))
        
        return x5    

In [9]:
class ModelSimple(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(259, 128, bias=True)
        self.linear2 = torch.nn.Linear( 128, 64, bias= True)
        self.linear3 = torch.nn.Linear( 64, 1, bias= True)
        
        self.linearTemp = torch.nn.Linear(128,1, bias=True)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.bn3 = torch.nn.BatchNorm1d(32)
        self.bn4 = torch.nn.BatchNorm1d(16)
        
        self.sig1 = torch.nn.Sigmoid()
        self.sig2 = torch.nn.Sigmoid()
        self.sig3 = torch.nn.Sigmoid()       
        self.sig4 = torch.nn.Sigmoid()
        self.sig5 = torch.nn.Sigmoid()
    
    def forward(self,x):
        x1 = self.bn1(F.relu(self.linear1(x)))
        x2 = self.bn2(F.relu(self.linear2(x1)))
        x3 = F.relu(self.linear3(x2))
        # x3 = self.linear3(x2)
        
        return x3
    
    def forwardSig(self,x):
        x1 = self.bn1( self.sig1(self.linear1(x)))
        x2 = self.bn2( self.sig2(self.linear2(x1)))
        x3 = self.sig3(self.linear3(x2))
        
        return x3    
    
    def forwardWithoutBN(self,x):
        x1 = F.relu(self.linear1(x))
        x2 = F.relu(self.linear2(x1))
        x3 = F.relu(self.linear3(x2))
        
        return x3
    

In [10]:
dataset = TensorDataset(X, Y)

In [11]:
dataloader = DataLoader(dataset, batch_size=5000,
                        shuffle=False, num_workers=4)  

In [12]:
#train_loader = torch.utils.data.DataLoader( dataset=X, 
 #                                          batch_size=batch_size, 
 #                                          shuffle=False))

In [13]:
model = Model()

In [14]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.ASGD(model.parameters(), lr=0.01)
#criterion = torch.nn.CrossMapLRN2d(259)
#loss_fn = torch.nn.MSELoss(reduction='sum')
#criterion = torch.nn.CrossEntropyLoss()
#loss = torch.nn.MSELoss()
#loss = torch.nn.BCELoss()
loss = torch.nn.BCEWithLogitsLoss()

y = Y
x = X

for step in range(31):

        # i = 0
        #for x in X:
       # y = Y[i]
        #i += 1
        #hypothesis = model(x)
        #model = Model()
        hypothesis = model(x)
        # cost/loss function
        # cost = -(y * torch.log(hypothesis) + (1 - y) * torch.log(1 - hypothesis)).mean()
        # cost = criterion( hypothesis, y )
        yy = torch.unsqueeze(y, 1)
        cost = loss( hypothesis, yy )
        #print( x )
        #print(index, cost.data.numpy() )
        
        # cost = criterion( hypothesis, Y.long().view(-1))
        # cons = loss_fn( hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(step, cost.data.numpy())

In [15]:
def PrintAccuracyForSig():
    # Accuracy computation
    predicted = (model(X).data >0.5).float()
    accuracy = (predicted == Y.data).float().mean()
    print("Accuracy: ", accuracy)

In [ ]:
def PrintAccuracy():
    # Accuracy computation
    predicted = (model(X).data >0).float()
    accuracy = (predicted.reshape(-1) == Y.data).float().mean()
    print("Accuracy: ", accuracy)

In [ ]:
#y = Y
#x = X

for step in range(10001):

    #i = 0
    #for x in X:
    #    y = Y[i]
    #    i += 1
    for index, (x, y) in enumerate(dataloader):    
        #hypothesis = model(x)
        #model = Model()
        hypothesis = model(x)
        # cost/loss function
        # cost = -(y * torch.log(hypothesis) + (1 - y) * torch.log(1 - hypothesis)).mean()
        # cost = criterion( hypothesis, y )
        y = torch.unsqueeze(y, 1)
        cost = loss( hypothesis, y )
        #print( x )
        #print(index, cost.data.numpy() )
        
        # cost = criterion( hypothesis, Y.long().view(-1))
        # cons = loss_fn( hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if step % 100 == 0:
        print(step, cost.data.numpy())
        PrintAccuracy()

0 0.6923727989196777
Accuracy:  tensor(0.5500)
100 0.6693254113197327
Accuracy:  tensor(0.6279)
200 0.6631410717964172
Accuracy:  tensor(0.6260)
300 0.657487154006958
Accuracy:  tensor(0.6278)
400 0.6512547731399536
Accuracy:  tensor(0.6285)
500 0.6406717896461487
Accuracy:  tensor(0.6237)
600 0.6338208913803101
Accuracy:  tensor(0.6288)
700 0.6333965063095093
Accuracy:  tensor(0.6279)
800 0.6478458642959595
Accuracy:  tensor(0.6274)
900 0.6294586658477783
Accuracy:  tensor(0.6291)
1000 0.622764527797699
Accuracy:  tensor(0.6275)
1100 0.619151771068573
Accuracy:  tensor(0.6292)
1200 0.6124328970909119
Accuracy:  tensor(0.6295)
1300 0.6223899126052856
Accuracy:  tensor(0.6304)
1400 0.606228768825531
Accuracy:  tensor(0.6307)
1500 0.6177878975868225
Accuracy:  tensor(0.6321)
1600 0.6213812232017517
Accuracy:  tensor(0.6291)
1700 0.6200570464134216
Accuracy:  tensor(0.6292)
1800 0.610107421875
Accuracy:  tensor(0.6323)
1900 0.5987026691436768
Accuracy:  tensor(0.6090)
2000 0.5908048152923

In [ ]:
# Accuracy computation
predicted = (model(X).data > 0).float()
accuracy = (predicted.reshape(-1) == Y.data).float().mean()
print("\nHypothesis: ", hypothesis.data.numpy(), "\nCorrect (Y): ", predicted.numpy(), "\nAccuracy: ", accuracy)

In [ ]:
pData = model(X)

In [ ]:
(predicted.data == 1).float().mean()

In [ ]:
(Y.data == 0).float().mean()

In [ ]:
( predicted == 0 ).float().mean()

In [ ]:
l = (predicted.data).float().tolist()

In [ ]:
predicted.tolist()

In [ ]:
loss(predicted,torch.unsqueeze(Y, 1))

In [ ]:
len(Y)

In [ ]:
len(predicted)

In [ ]:
res = model(X)

In [ ]:
res

In [ ]:
predicted

In [ ]:
tmp = model(X).data

In [ ]:
tmp

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(tmp.tolist())

In [ ]:
df.describe()

In [ ]:
len( df.loc[ df[0] < 0.1 ] )

In [ ]:
len(df)

In [ ]:
dfResult = pd.DataFrame( Y.tolist() )

In [ ]:
dfResult.loc[ df[0] < 0.1 ].sum()

In [ ]:
dfResult.loc[ df[0] < 0.1 ].describe()

In [ ]:
t = predicted==Y.data

In [ ]:
Y.data

In [ ]:
predicted

In [ ]:
Y.data.shape

In [ ]:
(predicted.reshape(26039)==Y.data).float().mean()

In [ ]:
predicted.reshape(-1)

In [ ]:
Y.data

In [ ]:
predicted